<a href="https://colab.research.google.com/github/nouranali/SureStart_VAILTraining/blob/master/Upsampling_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Today I'll be working on Upsampling2D and Conv2DTranspose layers on Keras as an action item applied to GANs

### let's start with a simple example

In [11]:
from numpy import asarray
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Conv2D

In [4]:
# input data
X = asarray([[1, 2],
			 [3, 4]])
# show input data for context
print(X)
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

[[1 2]
 [3 4]]


In [5]:
print(X)

[[[[1]
   [2]]

  [[3]
   [4]]]]


In [6]:
model = Sequential()
model.add(UpSampling2D(input_shape=(2, 2, 1)))
# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, 4, 4, 1)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [7]:
# make a prediction with the model
yhat = model.predict(X)
# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))
# summarize output
print(yhat)

[[1. 1. 2. 2.]
 [1. 1. 2. 2.]
 [3. 3. 4. 4.]
 [3. 3. 4. 4.]]


In [8]:
# use upsapling layer
model.add(UpSampling2D(size=(2, 3)))

In [9]:
#  using bilinear interpolation when upsampling instead of nearest neighbor
model.add(UpSampling2D(interpolation='bilinear'))

In [12]:
#create new model with upsampling 2D
model2 = Sequential()
# define input shape, output enough activations for for 128 5x5 image
model2.add(Dense(128 * 5 * 5, input_dim=100))
# reshape vector of activations into 128 feature maps with 5x5
model2.add(Reshape((5, 5, 128)))

In [13]:
model2.add(UpSampling2D())

In [14]:
model.add(Conv2D(1, (3,3), padding='same'))
# summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, 4, 4, 1)           0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 12, 1)          0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 16, 24, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 24, 1)         10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### Conv2D Transpose layer

In [15]:
from keras.models import Sequential
from keras.layers import Conv2DTranspose
# define input data
X = asarray([[1, 2],
			 [3, 4]])
# show input data for context
print(X)
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))
# define model
model = Sequential()
model.add(Conv2DTranspose(1, (1,1), strides=(2,2), input_shape=(2, 2, 1)))
# summarize the model
model.summary()
# define weights that they do nothing
weights = [asarray([[[[1]]]]), asarray([0])]
# store the weights in the model
model.set_weights(weights)
# make a prediction with the model
yhat = model.predict(X)
# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))
# summarize output
print(yhat)

[[1 2]
 [3 4]]
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 4, 4, 1)           2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
[[1. 0. 2. 0.]
 [0. 0. 0. 0.]
 [3. 0. 4. 0.]
 [0. 0. 0. 0.]]


In [16]:
...
# example of using padding to ensure that the output is only doubled
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same', input_shape=(2, 2, 1)))

### example Generator Model With the Conv2DTranspose Layer


In [17]:
model = Sequential()
# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))
# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))
# double input from 128 5x5 to 1 10x10 feature map
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same'))
# summarize model
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3200)              323200    
_________________________________________________________________
reshape_1 (Reshape)          (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________


So today I learnt about 2 layers, upsampling which increases input dimensions and followed by a conv2d layer to interpret the doubled input and translate it,
and Conv2DTranspose which is an inverse convolutional layer that will both upsample input and learn how to fill in details during the model training process.